# 类

-----------------------

类提供了把数据和功能绑定在一起的方法。创建新类时创建了新的对象```类型```，从而能够创建该类型的新```实例```。实例具有能维持自身状态的属性，还具有能修改自身状态的方法（由其所属的类来定义）。

Python的类支持面向对象编程（OOP）的所有标准特性：类的继承机制支持多个基类、派生的类能覆盖基类的方法、类的方法能调用基类中的同名方法。对象可包含任意数量和类型的数据。和模块一样，类也支持Python动态特性：在运行时创建，创建后还可以修改。

## 1.1 名称(name)和对象(object)

对象之间相互独立，多个名称（甚至是多个作用域内的多个名称）可以绑定到同一对象。这在其他语言中通常被称为别名。Python 初学者通常不容易理解这个概念，处理数字、字符串、元组等不可变基本类型时，可以不必理会。但是，对于涉及可变对象（如列表、字典，以及大多数其他类型）的 Python 代码的语义，别名可能会产生意料之外的效果。这样做，通常是为了让程序受益，因为别名在某些方面就像指针。例如，传递对象的代价很小，因为实现只传递一个指针；如果函数修改了作为参数传递的对象，调用者就可以看到更改——无需像 Pascal 那样用两个不同的机制来传参。



## 1.2 Python的作用域(scope)和命名空间(namespace)

在介绍类前，首先要介绍Python的作用域规则。类定义对命名空间有一些巧妙的技巧，了解作用域和命名空间的工作机制有利于加强对类的理解。

我们先了解一些定义:

namespace （命名空间）是从名称到对象的映射。现在，大多数命名空间都使用Python字典实现。命名空间的例子有：内置名称集合（包括 abs()函数以及内置异常的名称等）；一个模块的全局名称；一个函数调用中的局部名称。对象的属性集合也是命名空间的一种形式。关于命名空间的一个重要知识点是，不同命名空间中的名称之间绝对没有关系；例如，两个不同的模块都可以定义 ```maximize```函数，且不会造成混淆。用户使用函数时必须要在函数名前面加上模块名。

点号之后的名称是```属性```。例如，表达式```z.real```中，```real```是对象```z```的属性。严格来说，对模块中名称的引用是属性引用：表达式 ```modname.funcname```中，```modname```是模块对象，```funcname```是模块的属性。模块属性和模块中定义的全局名称之间存在直接的映射：它们共享相同的命名空间。

**属性可以是只读的或者可写的**。在后一种情况下，可以对属性进行赋值。 模块属性是可写的：你可以写入```modname.the_answer = 42```。也可以使用```del```语句删除可写属性。 例如，```del modname.the_answer```将从名为```modname```对象中移除属性```the_answer```。

命名空间是在不同时刻创建的，且拥有不同的生命周期。内置名称的命名空间是在Python解释器启动时创建的，永远不会被删除。模块的全局命名空间在读取模块定义时创建；通常，模块的命名空间也会持续到解释器退出。从脚本文件读取或交互式读取的，由解释器顶层调用执行的语句是```__main__```模块调用的一部分，也拥有自己的全局命名空间。内置名称实际上也在模块里，即```builtins```。

函数的局部命名空间在函数被调用时被创建，并在函数返回或抛出未在函数内被处理的异常时，被删除。（实际上，用“遗忘”来描述实际发生的情况会更好一些。）当然，每次递归调用都有自己的局部命名空间。

一个命名空间的```作用域```是Python代码中的一段文本区域，从这个区域可直接访问该命名空间。“可直接访问”的意思是，该文本区域内的名称在被非限定引用时，查找名称的范围，是包括该命名空间在内的。

作用域虽然是被静态确定的，但会被动态使用。执行期间的任何时刻，都会有 3或4个“命名空间可直接访问”的嵌套作用域：

- 最内层作用域，包含局部名称，并首先在其中进行搜索

- 那些外层闭包函数的作用域，包含“非局部、非全局”的名称，从最靠内层的那个作用域开始，逐层向外搜索。

- 倒数第二层作用域，包含当前模块的全局名称

- 最外层（最后搜索）的作用域，是内置名称的命名空间

如果一个名称被声明为全局，则所有引用和赋值都将直接指向“倒数第二层作用域”，即包含模块的全局名称的作用域。 要重新绑定在最内层作用域以外找到的变量，可以使用```nonlocal```语句；如果未使用```nonlocal```声明，这些变量将为只读（尝试写入这样的变量将在最内层作用域中创建一个新的局部变量，而使得同名的外部变量保持不变）。

通常，当前局部作用域将（按字面文本）引用当前函数的局部名称。在函数之外，局部作用域引用与全局作用域一致的命名空间：模块的命名空间。 类定义在局部命名空间内再放置另一个命名空间。

划重点，作用域是按字面文本确定的：模块内定义的函数的全局作用域就是该模块的命名空间，无论该函数从什么地方或以什么别名被调用。另一方面，实际的名称搜索是在运行时动态完成的。但是，Python正在朝着“编译时静态名称解析”的方向发展，因此不要过于依赖动态名称解析！（局部变量已经是被静态确定了。）

Python有一个特殊规定。如果不存在生效的```global```或```nonlocal```语句，则对名称的赋值总是会进入最内层作用域。赋值不会复制数据，只是将名称绑定到对象。删除也是如此：语句```del x```从局部作用域引用的命名空间中移除对 x 的绑定。所有引入新名称的操作都是使用局部作用域：尤其是 import 语句和函数定义会在局部作用域中绑定模块或函数名称。

```global```语句用于表明特定变量在全局作用域里，并应在全局作用域中重新绑定；```nonlocal```语句表明特定变量在外层作用域中，并应在外层作用域中重新绑定。

## 1.3 类详解

### 1.3.1 类定义语法

最简单的类定义形式如下：
```python
class ClassName:
    <statement-1>
    .
    .
    .
    <statement-N>
```

与函数定义的```def语句```一样，类定义必须先执行才能生效，这句话的意思是，如果把类定义放入if语句的分支里或函数内部中，只有当if语句的分支或函数被执行时才会生效，类才会被定义和实例化，同时，类定义的作用域仅限于包含它的分支或函数内部，在外部是无法被访问的。

In [1]:
def my_function():
    if True:
        class Myclass():
            pass
        obj = Myclass()

my_function()
obj = Myclass()

NameError: name 'Myclass' is not defined

在实践中，类定义内的语句通常都是函数定义，但也可以是其他语句。类里的函数定义一般是特殊的参数列表，这是由方法调用的约定范围所指明的。

当进入类定义时，将创建一个新的namespace，并将其用作局部作用域，因此，所有对局部变量的赋值都是在这个新的namespace之内。特别的，函数定义会绑定到这里的新函数名称。

当从结尾处正常离开类定义时，将创建一个```类```对象。这基本上是一个围绕类定义所创建的namespace的包装器。原始的（在进入类定义之前有效的）作用域将重新生效。类对象将在这里与类定义头所给出的类名称进行绑定。

In [2]:
# 例1 创建Dog类
class Dog(): # ①
    """一次模拟小狗的简单尝试""" # ②
    
    def __init__(self, name, age): # ③
        """初始化属性name和age"""
        self.name = name # ④
        self.age = age
        
    def sit(self): # ⑤
        """模拟小狗被命令时蹲下"""
        print(self.name.title() + " is now sitting.")
        
    def roll_over(self):
        """模拟小狗被命令时打滚"""
        print(self.name.title() + " rolled over!")

详解：

① 根据约定，在Python中，首字母大写的名称指的是**类**。这个定义的括号中是空的，因为我们要从空白创建这个类。

② 我们编写了一个文档字符串，对这个类的功能做了描述

③ 方法__init__():**类中的函数称为方法，有关函数的一切都适用于方法，目前而言唯一的差别是调用方式**。方法__init__()是一个特殊的方法，每当程序根据Dog类创建实例时，都会自动运行它。此方法前后的下划线是一种约定，旨在避免Python默认方法与普通方法发生名称冲突。在方法的定义中，形参**self**必不可少，还必须位于其他形参的前面。因为Python在运行方法__init__()来创建实例时，将自动传入实参self，**每个与类相关联的方法调用都自动传递实参self，他是一个指向实例本身的引用，让实例能够访问类中的属性和方法**，此例中，每当我们根据Dog类创建实例时，都只需给最后两个形参（name和age）提供值。**方法__init__()并未显式地包含return语句，但Python自动返回一个表示这条小狗的实例**

④该处定义的两个变量都有前缀self，以self为前缀的变量都可供类中的所有方法使用(见下例，如果有定义在方法中的，以self为前缀的变量，那么只有在该方法被调用后，其他的方法才能使用该变量)，我们还可以通过类的任何实例来访问这些变量。像这样可通过实例访问的变量被称为属性。

In [7]:
class Myclass():
    def __init__(self):
        self.a = 1
        self.b = 2

    def ptiny(self):
        self.c = 3

    def show_atr(self, ):
        print(self.a)
        print(self.c)

obj = Myclass()
obj.ptiny()
obj.show_atr()
    

1
3


### 1.3.2 Class对象

类对象支持两种操作：```属性引用```和```实例化```:

- **属性引用**使用Python中所有属性引用所使用的标准语法```obj.name```。有效的属性名称是类对象被创建时存在于类的namespace中的所有名称。

```python
>>>class Myclass:
        '''A simple example'''
        i = 123

        def f(slef):
            return 'hello world'

    obj = Myclass
    print(obj.i)
    obj.f
    obj.__doc__

123
<function __main__.Myclass.f(slef)>
'A simple example'
```

见上例，```Myclass.i```和```Myclass.f```就是有效的属性引用，将分别返回一个整数和一个函数对象。类属性也可以被赋值，因此可以通过更改赋值来更改Myclass.i的值，```__doc__```也是一个有效的属性，将返回所属类的文档字符串。


- **类的实例化**用函数表示法，可以把类对象视为返回值是该类的一个新实例的不带参数的函数。

### 1.3.3 实例对象

实例对象所能理解的唯一操作是**属性引用**。有两种有效的属性名称：```数据属性```和```方法```

- ```数据属性```不需要申明，就像局部变量一样，他们将在首次被赋值时产生。1.3.1中例，如果obj是类Myclass的一个实例，它具有两项初始的属性obj.a,obj.b，如果想要给实例obj添加一个在原始的类中不存在的属性，可以像给局部变量赋值一样创建新的属性，并在后续对该属性进行调用，最后可以通过del语句删除该属性，且不保留任何追踪信息。
```python
                    obj.counter = 0
                    .
                    .
                    del obj.counter
```

- 另一类实例属性引用称为```方法```，方法是从属于对象的函数。实例对象的有效方法名称依赖于其所属的类。根据定义，一个类中所有是函数对象的属性都是定义了其实例的相应方法。

### 1.3.4 方法对象

当一个方法被调用时会发生什么？方法的特殊之处就在于实例对象会作为函数的第一个参数被传入。调用一个具有n个参数的方法就相当于调用具有n+1个参数的函数，这个参数值为方法所属的实例对象，位置在其他参数之前。

In [23]:
class MyClassA():
    def __init__(self):
        self.a = 1
        self.b = 2

    def f(self, xyz):
        print(self.a, xyz)

x = MyClassA()
del x.b
try:
    print(x.b)
except AttributeError:
    y = MyClassA()
    print(y.b)

g = x.f
g(1)

2
1 1


### 1.3.5 类和实例变量

一般来说，实例变量(instance variable)用于每个实例的唯一数据，而类变量(class variable)用于类的所有实例共享的属性和方法。

```python
>>>class Dog():
        kind = 'canine'  # class variable
        
        def __init__(self, name):
            self.name = name     # instance variable
```

在实际操作过程中，我们应尽量少的使用类变量，因为当类变量是列表字典等可变数据类型时，如果我们在类中定义了操作类变量的函数，那么可能会导致意想不到的结果，因为一个类的所有实例都将只共享一个单独的变量。例如：

```python
>>>class Cat():
        tricks = []   # class variable
        
        def __init__(self, name):
            self.name = name

        def add_trick(self, trick_name):
            tricks.append(trick_name)

    a = Cat('Daisy')
    b = Cat('John')
    a.add_trick('roll over')
    b.add_trick('play dead')
    a.tricks

['roll over', 'play dead']
```

正确的类设计应该使用实例变量。

### 1.3.6 补充

如果同样的属性名称同时出现在实例和类中，则属性查找会优先选择实例。

方法的第一个参数常常被命名为```self```。这也不过就是一个约定:```self```这一名称在Python中绝对没有特殊含义。 但是要注意，不遵循此约定会使得你的代码对其他Python程序员来说缺乏可读性。

每个值都是一个对象，因此具有 类 （也称为 类型），并存储为 object.```__class__```。